In [19]:
#imports
import os
from PIL import Image
import torch
from torch.utils.data import Dataset, DataLoader, Subset
from torchvision import transforms
from sklearn.model_selection import train_test_split

In [20]:
class FingerprintDataset(Dataset):
    def __init__(self, image_dir, image_paths, labels, transform=None):
        self.image_dir = os.path.abspath(image_dir)
        self.transform = transform
        self.image_paths = image_paths
        self.labels = labels

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image_path = self.image_paths[idx]
        image = Image.open(image_path).convert("RGB")
        
        # Split the image into two fingerprints
        width, height = image.size
        left_fingerprint = image.crop((0, 0, width // 2, height))
        right_fingerprint = image.crop((width // 2, 0, width, height))
        
        if self.transform:
            left_fingerprint = self.transform(left_fingerprint)
            right_fingerprint = self.transform(right_fingerprint)
        
        label = self.labels[idx]
        return (left_fingerprint, right_fingerprint), label

def prepare_data(image_dir, test_size=0.2, random_state=42):
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])

    image_paths = []
    labels = []
    classes = ['class_0', 'class_1']

    for label, class_dir in enumerate(classes):
        class_path = os.path.join(image_dir, class_dir)
        if not os.path.exists(class_path):
            raise FileNotFoundError(f"The directory {class_path} does not exist.")
        for img_name in os.listdir(class_path):
            image_paths.append(os.path.join(class_path, img_name))
            labels.append(label)

    train_indices, test_indices = train_test_split(list(range(len(image_paths))), test_size=test_size,
                                                   random_state=random_state)

    train_data_info = (train_indices, image_paths, labels)
    test_data_info = (test_indices, image_paths, labels)

    return train_data_info, test_data_info

if __name__ == "__main__":
    image_dir = "C:/Users/FinalProject/Desktop/LoRA/Dino/OurData"  # Replace with your dataset directory
    print(f"Using dataset directory: {os.path.abspath(image_dir)}")
    train_data_info, test_data_info = prepare_data(image_dir)
    torch.save(train_data_info, 'train_data.pth')
    torch.save(test_data_info, 'test_data.pth')
    print("Data prepared and saved.")

Using dataset directory: C:\Users\FinalProject\Desktop\LoRA\Dino\OurData
Data prepared and saved.


In [21]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Subset
from torchvision import transforms
from transformers import AutoImageProcessor, AutoModelForImageClassification
from peft import LoraConfig, get_peft_model
from torch.utils.tensorboard import SummaryWriter

def get_classifier_in_features(model):
    for name, module in model.named_modules():
        if isinstance(module, nn.Linear):
            return module.in_features
    raise AttributeError("No linear layer found in the model.")

class SiameseDinoNetwork(nn.Module):
    def __init__(self, base_model):
        super(SiameseDinoNetwork, self).__init__()
        self.base_model = base_model
        # Access the underlying model's classifier in_features correctly
        in_features = get_classifier_in_features(self.base_model)
        self.fc = nn.Linear(in_features * 2, 1)  # Binary classification

    def forward(self, left_image, right_image):
        left_output = self.base_model(left_image).logits
        right_output = self.base_model(right_image).logits
        
        combined_output = torch.cat((left_output, right_output), dim=1)
        output = self.fc(combined_output)
        
        return torch.sigmoid(output).squeeze()

def load_data(train_path, test_path, image_dir):
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])

    train_indices, image_paths, labels = torch.load(train_path)
    test_indices, _, _ = torch.load(test_path)

    train_data = Subset(FingerprintDataset(image_dir, image_paths, labels, transform), train_indices)
    test_data = Subset(FingerprintDataset(image_dir, image_paths, labels, transform), test_indices)

    return train_data, test_data

# Load Training & Test DataSets
image_dir = "C:/Users/FinalProject/Desktop/LoRA/Dino/OurData"
train_data, test_data = load_data('train_data.pth', 'test_data.pth', image_dir)

# Set Device
os.environ["HF_HUB_DISABLE_SYMLINKS_WARNING"] = "1"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Hyperparameters
batch_size = 64
epochs = 5
learning_rate = 0.001

# Data Loader
trainloader = DataLoader(train_data, batch_size=batch_size, shuffle=True, num_workers=2)
testloader = DataLoader(test_data, batch_size=batch_size, shuffle=False, num_workers=2)

# Load a pre-trained model and feature extractor
processor = AutoImageProcessor.from_pretrained('facebook/dinov2-base')
base_model = AutoModelForImageClassification.from_pretrained('facebook/dinov2-base')

# Apply LoRA using the PEFT library
config = LoraConfig(
    r=16,
    lora_alpha=16,
    target_modules=["query", "value"],
    lora_dropout=0.1,
    bias="none",
    modules_to_save=["classifier"],
)
lora_model = get_peft_model(base_model, config)
lora_model.print_trainable_parameters()

# Freeze all the parameters in the base model except for LoRA parameters
for name, param in lora_model.named_parameters():
    if 'lora' not in name:
        param.requires_grad = False

# Create Siamese Network
model = SiameseDinoNetwork(lora_model).to(device)

# Define loss function and optimizer
criterion = nn.BCELoss()
optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9)
torch.cuda.empty_cache()

Some weights of Dinov2ForImageClassification were not initialized from the model checkpoint at facebook/dinov2-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


trainable params: 592,898 || all params: 87,176,452 || trainable%: 0.6801125606717741


In [ ]:
# Initialize TensorBoard
writer = SummaryWriter('runs/LoRA')

# Training The Model
for epoch in range(epochs):
    model.train()  # Set the model to training mode
    running_loss = 0.0

    for i, data in enumerate(trainloader, 0):
        (left_images, right_images), labels = data
        left_images, right_images, labels = left_images.to(device), right_images.to(device), labels.to(device).float()

        optimizer.zero_grad()
        outputs = model(left_images, right_images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    epoch_loss = running_loss / len(trainloader)

    model.eval()  # Set the model to evaluation mode
    correct = 0
    total = 0
    with torch.no_grad():
        for data in testloader:
            (left_images, right_images), labels = data
            left_images, right_images, labels = left_images.to(device), right_images.to(device), labels.to(device).float()
            outputs = model(left_images, right_images)
            predicted = (outputs > 0.5).float()
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
        epoch_accuracy = 100 * correct / total

        writer.add_scalar('Accuracy/test', epoch_accuracy, epoch)
        writer.add_scalar('Loss/train', epoch_loss, epoch)

        print(f'Accuracy : {epoch_accuracy} % ' + ' loss: %.3f' % (running_loss / len(trainloader)))

print('Finished Training')
writer.close()